<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/tensorflow/TensorFlow_RegressaoLinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow - Regressão Linear

## Carregando arquivo .csv

In [0]:
!wget vendas
!mkdir datasets
!mv *.csv datasets
!ls -ilah datasets

In [0]:
# Versão 1 do Modelo


import os
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Nível de log
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'

# ------------------------- Seção 1 - Carregando e Normalizando os Dados -------------------------

# Carrega dados de treino
dados_treino = pd.read_csv("datasets/vendas_data_training.csv", dtype = float)

# Define X e Y de treino
X_treino = dados_treino.drop("total_vendas", axis= 1).values
Y_treino = dados_treino[['total_vendas']].values

# Carrega dados de teste
dados_teste = pd.read_csv("datasets/vendas_data_test.csv", dtype = float)

# Define X e Y de teste
X_teste = dados_teste.drop("total_vendas", axis = 1).values
Y_teste = dados_teste[['total_vendas']].values

# Criando operadores de escala
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

# Aplicando escala aos dados de treino
X_scaled_treino = X_scaler.fit_transform(X_treino)
Y_scaled_treino = Y_scaler.fit_transform(Y_treino)

# Aplicando escala aos dados de teste
X_scaled_teste = X_scaler.transform(X_teste)
Y_scaled_teste = Y_scaler.transform(Y_teste)

# ------------------------- Seção 2 - Estrutura do Modelo -------------------------

# Hiperparâmetros
learning_rate = 0.001
num_epochs = 100
display_step = 5

# Definindo inputs e outputs
num_inputs = 9
num_outputs = 1

# Camadas
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

# ------------------------- Seção 3 - Construindo Camadas da Rede Neural -------------------------

# Camada de Input
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, num_inputs))

# Camada 1
with tf.variable_scope('layer1'):
    weights = tf.get_variable(name = 'weights1', shape=[num_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = 'biases1', shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Camada 2
with tf.variable_scope('layer2'):
    weights = tf.get_variable(name = 'weights2', shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = 'biases2', shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Camada 3
with tf.variable_scope('layer3'):
    weights = tf.get_variable(name = 'weights3', shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = 'biases3', shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Camada de Output
with tf.variable_scope('output'):
    weights = tf.get_variable(name = 'weights4', shape=[layer_3_nodes, num_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = 'biases4', shape=[num_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

# ------------------------- Seção 4 - Custo e Otimização -------------------------

# Custo
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, num_outputs))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

# Otimizador
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# ------------------------- Seção 5 - Sessão TensorFlow -------------------------

# Abrindo Sessão TensorFlow
with tf.Session() as session:

    # Inicialização das Variáveis
    session.run(tf.global_variables_initializer())

    # Loop for pelas epochs
    for epoch in range(num_epochs):

        # Treinamento
        session.run(optimizer, feed_dict={X:X_scaled_treino, Y:Y_scaled_treino})

        # Progresso do Treinamento
        if epoch % display_step == 0:
            training_cost = session.run(cost, feed_dict={X:X_scaled_treino, Y:Y_scaled_treino})
            test_cost = session.run(cost, feed_dict={X: X_scaled_teste, Y: Y_scaled_teste})
            print("\nCusto  em Treinamento: {}".format(training_cost))
            print("Custo  em Teste: {}".format(test_cost))

    # Print
    print("\nTreinamento Concluído")

    final_training_cost = session.run(cost, feed_dict={X: X_scaled_treino, Y: Y_scaled_treino})
    final_test_cost = session.run(cost, feed_dict={X: X_scaled_teste, Y: Y_scaled_teste})

    print("\nCusto Final em Treinamento: {}".format(final_training_cost))
    print("Custo Final em Teste: {}".format(final_test_cost))

    # Variável com valores previstos normalizados
    Y_predicted_scaled = session.run(prediction, feed_dict={X:X_scaled_teste})

    # Remove a escala
    Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)

    # Coleta os dados reais e os valores previstos
    total_vendas_real = dados_teste['total_vendas'].values[1]
    total_vendas_previsto = Y_predicted[1][0]

    # Print do resultado
    print("\nTotal Vendas Real de 1 Seguro: {}".format(total_vendas_real))
    print("Total Vendas Previsto de 1 Seguro: {}".format(total_vendas_previsto))
    print("Erro = : {}".format(total_vendas_real - total_vendas_previsto))